In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip install seqeval
!pip install accelerate -U


You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 290 kB 5.0 MB/s eta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang,cache_dir='.cache')

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [14]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features

print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [15]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [16]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [17]:
!pip install protobuf

You should consider upgrading via the '/data/sknigam/shivammis23/virtual/bin/python3 -m pip install --upgrade pip' command.


In [18]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner_tags')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )

In [19]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly
model=model.to("cuda:0")

In [20]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=128,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
train_dataset = raw_dataset["train"]
train_dataset = train_dataset.select(range(20000))  # Select only the first 30,000 lines
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=16,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [22]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=16,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)


Running tokenizer on Validation dataset (num_proc=16):   0%|          | 0/13460 [00:00<?, ? examples/s]

In [23]:
test_dataset = raw_datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=16,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset (num_proc=16):   0%|          | 0/867 [00:00<?, ? examples/s]

In [24]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [25]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

/tmp/ipykernel_439024/3673332548.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/data/sknigam/shivammis23/virtual/lib/python3.8/site-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [31]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    num_train_epochs=2.0,
    evaluation_strategy='epoch',
    fp16=True,
    accelerator_config={'split_batches': True, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
)

In [32]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': True, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_la

In [38]:
train_result = trainer.train()
metrics = train_result.metrics

OutOfMemoryError: CUDA out of memory. Tried to allocate 312.00 MiB. GPU 1 has a total capacity of 39.39 GiB of which 201.19 MiB is free. Process 3226211 has 35.45 GiB memory in use. Process 294322 has 3.33 GiB memory in use. Including non-PyTorch memory, this process has 416.00 MiB memory in use. Of the allocated memory 16.00 KiB is allocated by PyTorch, and 1.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def save_metrics_to_file(metrics, file_path):
    with open(file_path, 'w') as f:
        for key, value in metrics.items():
            f.write(f"{key}: {value}\n")

if __name__ == "__main__":
    metrics = trainer.evaluate()
    trainer.log_metrics("eval", metrics)
    metrics_file = "eval_metrics.txt"
    save_metrics_to_file(metrics, metrics_file)


In [54]:
test_dataset = raw_datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=16,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset (num_proc=16):   0%|          | 0/867 [00:00<?, ? examples/s]

In [55]:
tokenized_test_set = {}

for lang in raw_datasets:
  tokenized_test_set[lang] = raw_datasets['test'].map(
      tokenize_and_align_labels,
      batched=True,
      num_proc=8,
      load_from_cache_file=True,
      desc="Running tokenizer on test dataset of language {0}".format(lang),
  )

Running tokenizer on test dataset of language train (num_proc=8):   0%|          | 0/867 [00:00<?, ? examples/…

In [41]:
final_metrics = {}

for lang in tokenized_test_set:
  predictions, labels, metrics = trainer.predict(tokenized_test_set[lang], metric_key_prefix=lang)

  lang_specific_results = {}
  for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]
  final_metrics[lang] = lang_specific_results

OutOfMemoryError: CUDA out of memory. Tried to allocate 312.00 MiB. GPU 1 has a total capacity of 39.39 GiB of which 201.19 MiB is free. Process 3226211 has 35.45 GiB memory in use. Process 294322 has 3.33 GiB memory in use. Including non-PyTorch memory, this process has 416.00 MiB memory in use. Of the allocated memory 24.00 KiB is allocated by PyTorch, and 1.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [101]:
import pandas as pd

combined_results = pd.DataFrame.from_dict(
            final_metrics, orient="index"
        )

print(combined_results)
combined_results.to_csv("test_metrics.csv")

            Precision    Recall        F1
train        0.029666  0.065974  0.040928
test         0.029666  0.065974  0.040928
validation   0.029666  0.065974  0.040928


In [ ]:
model.save_pretrained('my_indicNER')

In [ ]:
tokenizer.save_pretrained("tokenizer_indicNER")

In [42]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [43]:
import json
import torch

In [44]:
config = json.load(open("my_indicNER/config.json"))

In [45]:
config["id2label"] = id2label
config["label2id"] = label2id


In [46]:
json.dump(config, open("my_indicNER/config.json","w"))

In [47]:
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("my_indicNER")

In [48]:
def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model_fine_tuned(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model_fine_tuned.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        for index in range(len(sentence.split(' '))):
            if(index<len(predicted_labels)):
                ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
            else:
                ner_output.append((sentence.split(' ')[index], 'O'))
        return ner_output


In [49]:

sentences = []
with open('ground_truth_words_only.txt', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip(" ")
        sentences.extend(line.split('\n'))

# Process each sentence and get NER results
ner_results = []
for sentence in sentences:
    if sentence.strip() == "":
        continue
    result = get_ner(sentence)
    ner_results.append(result)

print(ner_results)

# print(ner_results)

[[('सरकार', 'O'), ('ने', 'O'), ('विभिन्न', 'O'), ('खेलों', 'O'), ('के', 'O'), ('इन', 'O'), ('खिलाडिय़ों', 'O'), ('में', 'O'), ('से', 'O'), ('चार', 'O'), ('को', 'O'), ('सीधे', 'O'), ('पुलिस', 'O'), ('उपाधीक्षक', 'O'), ('(डीएसपी),', 'O'), ('18', 'B-ORG'), ('को', 'O'), ('निरीक्षक', 'O'), ('(इंस्पेक्टर)', 'O'), ('तथा', 'O'), ('14', 'O'), ('को', 'O'), ('उप-निरीक्षक', 'O'), ('(सब-इंस्पेक्टर)', 'O'), ('नियुक्त', 'O'), ('किया', 'O'), ('है।', 'O')], [('मिस्र', 'B-LOC'), ('की', 'O'), ('जनता', 'O'), ('ने', 'O'), ('यह', 'O'), ('दिखा', 'O'), ('दिया', 'O'), ('कि', 'O'), ('वह', 'O'), ('प्रजातंत्र', 'O'), ('की', 'O'), ('हामी', 'O'), ('है', 'O'), ('और', 'O'), ('उसने', 'O'), ('इस', 'O'), ('मिथक', 'O'), ('को', 'O'), ('भी', 'O'), ('तोड़', 'O'), ('दिया', 'O'), ('कि', 'O'), ('मुसलमान', 'O'), ('एक', 'O'), ('हिंसक', 'O'), ('कौम', 'O'), ('हैं', 'O'), ('और', 'O'), ('उन्हें', 'O'), ('प्रजातंत्र', 'O'), ('में', 'O'), ('कोई', 'O'), ('दिलचस्पी', 'O'), ('नहीं', 'O'), ('है.', 'O')], [('रुद्राक्ष', 'O'), ('की', 'O'), ('

In [51]:
with open('indicNER-prediction.txt', 'r', encoding='utf-8') as file:
    # Read the contents of the file
    sample_data = file.read()
    
# Parse the sample data string
parsed_data = eval(sample_data)

# Initialize an empty list to store the second values from each pair in all lists
second_values_list = []

# Iterate through each list of pairs in the parsed data
for sublist in ner_results:
    # Initialize an empty list to store the second values from pairs in the current sublist
    second_values_sublist = []
    
    # Iterate through each pair in the current sublist
    for pair in sublist:
        # Extract the second value from the current pair and append it to the sublist
        second_value = pair[1]
        second_values_sublist.append(second_value)
    
    # Append the sublist containing second values to the main list
    second_values_list.append(second_values_sublist)

# Print the resulting list of lists
print(second_values_list)


[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O'

In [52]:
# Write the resulting list of lists to a text file
with open('indicNER-tags.txt', 'w') as file:
    for sublist in second_values_list:
        file.write(','.join(sublist) + '\n')

print("Result has been saved in 'indicNER-tags.txt'")

Result has been saved in 'indicNER-tags.txt'


In [53]:
from sklearn.metrics import classification_report, accuracy_score

def read_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        data = []
        for line in lines:
            elements = line.strip().split(',')
            data.append(elements)
    return data

# Example usage:
p1 = 'ground_truth_tags_only.txt'  # Update with your file path
ground = read_file(p1)
p2 = 'indicNER-tags.txt'  # Update with your file path
prediction = read_file(p2)
# print(ground)
# print("--------------------------------------------------------------------------------------------")
# print(prediction)
# print("-----------------------------------------------------------------------------------------------------------------------------")

true_labels = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC','B-MISC','I-MISC']

# Convert each element to string
true_label = [str(label) for true_label in ground for label in true_label]
predicted_label = [str(label) for predicted_label in prediction for label in predicted_label]

# print(true_label)
# print("-----------------------------------")
# print(predicted_label)



# Determine the minimum length between the two datasets
min_length = min(len(true_label), len(predicted_label))

# Truncate both datasets to the minimum length
true_label = true_label[:min_length]
predicted_label = predicted_label[:min_length]



# Calculate classification report
report = classification_report(true_label, predicted_label, labels=true_labels)

# Calculate overall accuracy
overall_accuracy = accuracy_score(true_label, predicted_label)

# Print the classification report
print(report)

# Print overall accuracy
print("Overall Accuracy:", overall_accuracy)

              precision    recall  f1-score   support

           O       0.91      0.98      0.94       470
       B-PER       0.20      0.33      0.25         3
       I-PER       0.50      0.50      0.50         2
       B-ORG       0.00      0.00      0.00         6
       I-ORG       0.00      0.00      0.00         5
       B-LOC       0.14      0.09      0.11        11
       I-LOC       0.00      0.00      0.00         3
      B-MISC       0.00      0.00      0.00        20
      I-MISC       0.00      0.00      0.00         3

   micro avg       0.88      0.89      0.88       523
   macro avg       0.19      0.21      0.20       523
weighted avg       0.82      0.89      0.85       523

Overall Accuracy: 0.8787878787878788


/data/sknigam/shivammis23/virtual/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/sknigam/shivammis23/virtual/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/sknigam/shivammis23/virtual/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)